# CO407H - Medical Image Computing

## Coursework - Age regression from brain MRI

Predicting the age from a brain MRI scan is believed to have diagnostic value in the context of a number of pathologies that cause structural changes and damage to the brain. Assuming an accurate predictor of brain age can be trained based on a set of healthy subjects, the idea is then to compare the predicted age obtained on a new patient scan with the real age of that patient. Discrepancy between predicted and real age might indicate the presence of pathology and abnormal changes to the brain.

The objective for the coursework is to implement two different supervised learning approaches for age regression from brain MRI data. Data from 652 subjects will be provided. Each approach will require a processing pipeline with different components that you will need to implement using methods that were discussed in the lectures and tutorials. There are dedicated sections in the Jupyter notebook for each approach which contain some detailed instructions and some hints and notes.

For many tasks, you will find useful ideas and implementations in the tutorial notebooks. Make sure to add documentation to your code. Markers will find it easier to understand your reasoning when sufficiently detailed comments are provided in your implementations.

#### Read the descriptions and provided code cells carefully and look out for the cells marked with 'TASK'.

In [1]:
# Use full browser width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Getting started

The following cells provide some helper functions to load the data, and provide some overview and visualisation of the statistics over the population of 652 subjects. Let's start by loading the meta data, that is the data containing information about the subject IDs, their age, and gender.

In [2]:
import pandas as pd
import os

# WARNING: Change directories as expected!
%cd '/home/wizofe'
data_dir = 'mic/project-data/'
meta_data = pd.read_csv(os.path.join(data_dir,'meta/clean_participant_data.csv'))
meta_data.head() # show the first five data entries

Let's have a look the the population statistics.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
plt.set_cmap('gist_earth')
plt.xkcd()

sns.catplot(x="gender_text", data=meta_data, kind="count")
plt.title('Gender distribution')
plt.xlabel('Gender')
plt.show()

sns.distplot(meta_data['age'], bins=[10,20,30,40,50,60,70,80,90])
plt.title('Age distribution')
plt.xlabel('Age')
plt.show()

plt.scatter(range(len(meta_data['age'])),meta_data['age'], marker='.')
plt.grid()
plt.xlabel('Subject')
plt.ylabel('Age')
plt.show()

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk

from ipywidgets import interact, fixed
from IPython.display import display

%matplotlib inline

# Calculate parameters low and high from window and level
def wl_to_lh(window, level):
    low = level - window/2
    high = level + window/2
    return low,high

def display_image(img, x=None, y=None, z=None, window=None, level=None):
    # Convert SimpleITK image to NumPy array
    img_array = sitk.GetArrayFromImage(img)
    
    # Get image dimensions in millimetres
    size = img.GetSize()
    spacing = img.GetSpacing()
    width  = size[0] * spacing[0]
    height = size[1] * spacing[1]
    depth  = size[2] * spacing[2]
    print(width, height, depth)
    
    if x is None:
        x = np.floor(size[0]/2).astype(int)
    if y is None:
        y = np.floor(size[1]/2).astype(int)
    if z is None:
        z = np.floor(size[2]/2).astype(int)
    
    if window is None:
        window = np.max(img_array) - np.min(img_array)
    
    if level is None:
        level = window / 2 + np.min(img_array)
    
    low,high = wl_to_lh(window,level)

    # Display the orthogonal slices
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

    ax1.imshow(img_array[z,:,:], cmap='gray', clim=(low, high), extent=(0, width, height, 0))
    ax2.imshow(img_array[:,y,:], origin='lower', cmap='gray', clim=(low, high), extent=(0, width,  0, depth))
    ax3.imshow(img_array[:,:,x], origin='lower', cmap='gray', clim=(low, high), extent=(0, height, 0, depth))

    # Additionally display crosshairs
    ax1.axhline(y * spacing[1], lw=1)
    ax1.axvline(x * spacing[0], lw=1)
    
    ax2.axhline(z * spacing[2], lw=1)
    ax2.axvline(x * spacing[0], lw=1)
    
    ax3.axhline(z * spacing[2], lw=1)
    ax3.axvline(y * spacing[1], lw=1)

    plt.show()
    
def interactive_view(img):
    size = img.GetSize() 
    img_array = sitk.GetArrayFromImage(img)
    interact(display_image,img=fixed(img),
             x=(0, size[0] - 1),
             y=(0, size[1] - 1),
             z=(0, size[2] - 1),
             window=(0,np.max(img_array) - np.min(img_array)),
             level=(np.min(img_array),np.max(img_array)));

In [5]:
def preprocess_brain(img, msk):
    """ From a given image and brain mask, skull strip the brain image and apply normalisation
    Args:
        img(sitk image): The original brain image including the skull
        msk(sitk image): A defined brain mask image
        
    Returns:
        new_brain(sitk image): The resulting skull stripped brain image
    """
    img_array = sitk.GetArrayFromImage(img)
    mask_array = sitk.GetArrayFromImage(msk)

    dump_skull = img_array
    
    # skull-strip
    dump_skull[mask_array==0] = 0
    # display_image(sitk.GetImageFromArray(dump_skull))
    
    # If a SITK Image Object is needed
    # new_brain = sitk.GetImageFromArray(dump_skull)
    # new_brain.CopyInformation(img)
    
    # normalise the image - zscore
    dump_skull = (dump_skull - np.mean(dump_skull[mask_array>0])) / np.std(img_array[mask_array>0])
    
    # create an unstructured/flat array with the intensities of the brain voxels
    X = dump_skull[mask_array>0].flatten().reshape(-1,1)
    n_pts = len(X.flatten())
    
    # set the sampling rate for a random subset
    sampling = 0.15
    X_subset = np.random.choice(X.flatten(),int(n_pts*sampling)).reshape(-1, 1)
    
    # Visualise for debugging purposes
    #display_image(skullstripped, window=400, level=200)

    return dump_skull, mask_array, X, X_subset

In [6]:
import matplotlib.mlab as mlab
from nilabels.tools.image_colors_manipulations.relabeller import relabeller
import sklearn.mixture as mixture
import scipy


def plot_gmm(x, gmm):
    omega = gmm.weights_
    mu = gmm.means_
    sigma = np.sqrt(gmm.covariances_)
    for ind in range(0,omega.shape[0]): 
        plt.plot(x,omega[ind]*scipy.stats.norm.pdf(x, mu[ind], sigma[ind]), linewidth=2, label='GMM Component '+str(ind))


def gmm_init(ref_patient_ID):
    """ Initialise a GMM using a base reference patient
    """
    # Initialise GMM with a reference patient
    img = sitk.ReadImage(glob.glob(os.path.join(data_dir,'images/')+'*'+ref_patient_ID+'*')[0])
    msk = sitk.ReadImage(glob.glob(os.path.join(data_dir,'masks/')+'*'+ref_patient_ID+'*')[0])

    just_brain, mask_array, X, X_subset = preprocess_brain(img,msk)
    n_clusters = 3
    
    gmm = mixture.GaussianMixture(n_components=n_clusters)
    gmm.fit(X_subset)
    
    return np.array(gmm.means_).reshape(-1,1)
    
        
def segment_brain(skullstripped, msk_array, X, X_subset, m_init, seg_file):
    """ Segment a skullstripped brain and save it to a file
    Args:
        skullstripped(numpy array): The skullstripped image in a numpy array format
        msk_array(numpy array): The given mask image
        X(numpy array): The flattened version of the skullstripped image array
        X_subset(numpy array): A randomly sampled subset of the skullstripped image
        segmentation_file(path): The path to save the segmentation
    Returns:
        seg_gmm (numpy array): The resulting labelled segmented image
    """

    # WM,GM,CSF tissue classes
    n_clusters = 3
    
    gmm = mixture.GaussianMixture(n_components=3, covariance_type = 'full', 
                                  max_iter = 1000, random_state = 0, means_init=m_init, warm_start = True)
    gmm.fit(X_subset)
    y = gmm.predict(skullstripped.flatten().reshape(-1, 1))
    
    # shift labels and ensure no background
    y = y + 1 
    y[(msk_array == 0).flatten()] = 0

    lab_array = y.reshape(skullstripped.shape).astype('uint8')
    seg_gmm = sitk.GetImageFromArray(lab_array)
    seg_gmm.CopyInformation(img)
    
    #display_image(sitk.LabelToRGB(seg_gmm))
    sitk.WriteImage(seg_gmm, seg_file)
    #print('Wrote file!')

    lim_low = np.min(X).astype(np.int)
    lim_high = np.max(X).astype(np.int)
    num_bins = 400

    return seg_gmm

# Histogram GMM plot
#     plt.figure(figsize=(10, 4), dpi=100)
#     plt.hist(X, bins=num_bins, density=True, range=(lim_low, lim_high), label='Intensity histogram', color='tomato');
#     x = np.linspace(lim_low,lim_high,num_bins).reshape(-1,1)
#     plot_gmm(x,gmm)
#     plt.plot(x,np.exp(gmm.score_samples(x)), linewidth=1.5, color='k', label='Gaussian Mixture Model')
#     plt.xlim([lim_low,lim_high])
#     plt.legend(loc=0, shadow=True, fontsize=10)

In [7]:
import pandas as pd 

# Shape statistics filter (for volume information)
global shape_stats 
shape_stats = sitk.LabelShapeStatisticsImageFilter()

# create a pandas dataframe
global df_main
df_main = pd.DataFrame()

def store_features(seg):
    """ Store image brain shape statistics in a csv file (using pandas DF)
    Args:
        seg(sitk image): The segmentation image in SITK Image object format
        df_main(pandas dataframe): The main pandas dataframe for feature saving
        shape_stats(sitk object): The feature statistics object from SITK
    Returns:
        df_main(pandas dataframe): The final dataframe with all the features
    """
    # Other features such as intensity statistics can also be extracted
    # intensity_stats = sitk.LabelIntensityStatisticsImageFilter()
    shape_stats.Execute(seg)
    
    stats_list = [(shape_stats.GetPhysicalSize(i)) 
                  for i in shape_stats.GetLabels()]
    total_volume = np.sum(stats_list)  # Compute total volume to obtain the volume ratio for each structure
    
    cols = ['Volume ratio']
    data = {'Total Volume':total_volume}
    vol_df = pd.DataFrame(data, index=[0])  
    stats_list = stats_list/total_volume
    
    stats = pd.DataFrame(data=stats_list, index=shape_stats.GetLabels(), 
                         columns=cols)
    stats_v = pd.concat([stats.iloc[0], stats.iloc[1], 
                         stats.iloc[2], vol_df.iloc[0]], axis = 0)
    
    df = pd.DataFrame(stats_v.values.reshape(1, len(stats_list)+1), 
                      columns = stats_v.index)
    df_main.append(df)
    
    return df_main

### TASK A-2: Feature calculation

Implement a function that calculates volume features given the three tissue volumes and the overal brain volume (which can be calculated from the brain masks). You should use this function to construct a big matrix $X$ with a row for each subject and features across the columns.

*Note:* You may need to experiment here what the best set of features is. Start with just calculating three simple features of relative tissue volumes. So you should initially construct an $X^{652 \times 3}$.

In [9]:
import glob
import os

# Initialise the GMM with a reference patient

ref_patient_ID = 'CC110037'
m_init = gmm_init(ref_patient_ID)

# Remove all the comments if display is desirable
# Disabled by default as for data limits on Jupyter Notebooks

for i in range(meta_data['ID'].count()):
    # Subject with index 0
    ID = meta_data['ID'][i]
    age = meta_data['age'][i]

    # Data folders
    image_dir = os.path.join(data_dir, 'images/')
    image_filenames = glob.glob(image_dir + '*.nii.gz')

    mask_dir = os.path.join(data_dir, 'masks/')
    mask_filenames = glob.glob(mask_dir + '*.nii.gz')

    greymatter_dir = os.path.join(data_dir, 'greymatter/')
    greymatter_filenames = glob.glob(greymatter_dir + '*.nii.gz')

    image_filename = [f for f in image_filenames if ID in f][0]
    img = sitk.ReadImage(image_filename)

    mask_filename = [f for f in mask_filenames if ID in f][0]
    msk = sitk.ReadImage(mask_filename)

    greymatter_filename = [f for f in greymatter_filenames if ID in f][0]
    gm = sitk.ReadImage(greymatter_filename)

    print('Imaging data of subject ' + ID + ' with age ' + str(age))

    print('\nMR Image (used in part A)')
    #display_image(img, window=400, level=200)
    
    print('Brain mask (used in part A)')
    #display_image(msk)

    #print(image_filename)
    print('Spatially normalised grey matter maps (used in part B)')
    #display_image(gm)

    # Ensure the registrations directory is created
    # if not exists
    rd = os.path.join(data_dir, 'segm/')
    if not os.path.exists(rd):
        os.makedirs(rd)

    # Storage location for the segmented file
    seg_file = os.path.join(rd, os.path.basename(ID + '.nii.gz'))
 
    # Preprocess the brain image
    just_brain, mask_array, X, X_subset = preprocess_brain(img,msk)
    seg_im = segment_brain(just_brain, mask_array, X, X_subset, m_init, seg_file)
    
    # Store the volumes in a CSV file
    sf = store_features(seg_im)
    
    display_image(sitk.GetImageFromArray(just_brain))
    display_image(sitk.LabelToRGB(seg_im))

#     # Plot the histogram before and after normalisation
#     plt.hist(skullstripped, bins=200, density=True)
#     plt.figure()
#     plt.hist((skullstripped-np.mean(skullstripped)) / np.std(skullstripped), bins=200, density=True)

# Store the features to a csv file 
sf.to_csv(os.path.join(data_dir, 'features.csv'), index = 0)

# A reminder, useful for Jupyter notebooks.
print('Finished Processing!')

In [8]:
def resample_image(image, out_spacing=(1.0, 1.0, 1.0), out_size=None, is_label=False, pad_value=0):
    """Resamples an image to given element spacing and output size.
    Modified from the KiTS19-Challenge, original code by Junqiang Chen (public domain)
    """

    original_spacing = np.array(image.GetSpacing())
    original_size = np.array(image.GetSize())

    if out_size is None:
        out_size = np.round(np.array(original_size * original_spacing / np.array(out_spacing))).astype(int)
    else:
        out_size = np.array(out_size)

    original_direction = np.array(image.GetDirection()).reshape(len(original_spacing),-1)
    original_center = (np.array(original_size, dtype=float) - 1.0) / 2.0 * original_spacing
    out_center = (np.array(out_size, dtype=float) - 1.0) / 2.0 * np.array(out_spacing)

    original_center = np.matmul(original_direction, original_center)
    out_center = np.matmul(original_direction, out_center)
    out_origin = np.array(image.GetOrigin()) + (original_center - out_center)

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size.tolist())
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(out_origin.tolist())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(pad_value)

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(image)

In [9]:
from collections import Counter
from collections import defaultdict

# set the voxels to isotropic resolution of 8mm
img_size = [64, 64, 64]
img_res = [8, 8, 8]

age = list(meta_data['age'])

# The total volumns, normalised and age-related
# ageVolumn = {age: [csf,gm,wm]} (dictionary of ages to CSF/GM/WM)
ageVolumn = defaultdict(np.array)
volTotal = img_res[0]**2 + img_res[1]**2+ img_res[2]**2
vols_normalised = np.zeros((3, meta_data['ID'].count()))

for i in range(meta_data['ID'].count()):
    ID = meta_data['ID'][i]
    img = sitk.ReadImage(os.path.join(rd, os.listdir(rd)[i]))
    seg = resample_image(img, img_res, img_size)

    img_array = sitk.GetArrayFromImage(img) 
    seg_array = sitk.GetArrayFromImage(seg)

    frq = Counter(seg_array.flatten()) 
    unit = img_res[0] * img_res[1] * img_res[2]
    vols_normalised[:,i] = np.array([frq[1]*unit/volTotal, frq[2]*unit/volTotal,frq[3]*unit/volTotal])
    
    ageVolumn[meta_data['age'][i]] = ageVolumn.get(meta_data['age'][i], np.array([0,0,0])) + np.array([frq[1]*unit, frq[2]*unit, frq[3]*unit])

# Plot the age against the normalised volume data
# plt.xkcd()
plt.figure()
plt.title('Features vs age', fontsize=14)
plt.xlabel('Age', fontsize = 14)
plt.ylabel('Relative normalised volume', fontsize = 14)
plt.legend(['CSF', 'GM', 'WM'], fontsize = 12)

plt.scatter(age,vols_normalised[0,:])
plt.scatter(age,vols_normalised[1,:])

plt.scatter(age,vols_normalised[2,:])

In [10]:
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import tree

import warnings

# SKLearn returns a lot of deprecated warnings, disable
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)


def scatter_plot(title, y_test, y_pred):
    """ Plot true labels vs. predicted labels 
    Args:
        title (str): The title of the plot
        y_test (numpy array): The test values of the data set
        y_pred (numpy array): The predictions fitted by the model
    
    Return:
        It displays the requested plot
    """
    
    plt.xkcd() # Some fun
    
    plt.figure()
    plt.title(title, fontsize=13)
    
    # Thanks to https://stackoverflow.com/a/34004236
    plt.rcParams['xtick.labelsize'] = 9
    plt.rcParams['ytick.labelsize'] = 9

    plt.scatter(y_test, y_pred)
    plt.scatter(y_test, y_test, c='xkcd:bubblegum')
    
    plt.xlabel('True Age', fontsize = 11)
    plt.ylabel('Predicted Age', fontsize = 12)
    
    plt.legend(['Model', 'True'], fontsize = 9, loc='upper left')
    plt.show()


def train_linear_regression(X_train, y_train, X_test, y_test):
    """ Train a Linear Regression Classifier
    Args:
        X_train (numpy array): input volume data
        y_train (numpy array): input age training data
        X_test (numpy array): volume test data
        y_test (numpy array): age test data
    Returns:
        model (object): A Linear Regression Model object
        R2L (float): The model R2 score
        RMSEL (float): The model root mean square error
    """
    R2L = []
    RMSEL = []
    
    LinearModel = linear_model.LinearRegression()
    LinearModel.fit(X_train, y_train.ravel())
  
    R2 = LinearModel.score(X_test, y_test)
    R2L.append(R2)   
    
    y_pred = LinearModel.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSEL.append(rmse)
    
    scatter_plot('Linear Regression', y_test, y_pred)
    
    return LinearModel, np.around(np.mean(R2L),decimals=4)*100, np.mean(RMSEL)


def train_bayessian_regression(X_train, y_train, X_test, y_test):
    """ Train a Bayessian Regression Classifier
    Args:
        X_train (numpy array): input volume data
        y_train (numpy array): input age training data
        X_test (numpy array): volume test data
        y_test (numpy array): age test data
    Returns:
        model (object): A Linear Regression Model object
        R2B (float): The model R2 score
        RMSEB (float): The model root mean square error
    """
    RMSEB = []
    R2B = []
    
    BayesianModel = linear_model.BayesianRidge()
    BayesianModel.fit(X_train, y_train.ravel())
    
    #score_Br.append(BayesianModel.score(X_test,y_test))
    y_true = y_test
    y_pred = BayesianModel.predict(X_test)
    
    #mean_error_Br.append(mean_absolute_error(y_true, y_pred))
    RMSEB.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    R2B.append(r2_score(y_true, y_pred))

    scatter_plot('Bayesian Regression', y_test, y_pred)
    
    return BayesianModel, np.around(np.mean(R2B),decimals=4)*100, np.mean(RMSEB)
    
# Fit and evaluate SVR with RBF kernel model. 
# Use grid search to find the best combination of hyperparameters. 
def train_SVR_RBF(X_train, y_train, X_test, y_test):
    """ Train a Bayessian Regression Classifier
    Args:
        X_train (numpy array): input volume data
        y_train (numpy array): input age training data
        X_test (numpy array): volume test data
        y_test (numpy array): age test data
    Returns:
        model (object): A Linear Regression Model object
        R2_SVR (float): The model R2 score
        RMSE_SVR(float): The model root mean square error
    """
    R2_SVR = []
    RMSE_SVR = []
    
    svr = svm.SVR(kernel = 'rbf')
    param_grid = {"epsilon": np.logspace(-10, 5, 16), "C": [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2],
                  "gamma": np.logspace(-10, 5, 16)}
    SVRModel = GridSearchCV(svr, cv=2, scoring='neg_mean_squared_error', param_grid=param_grid)

    # grid search to find the best h-parameteres
    SVRModel.fit(X_train,y_train.ravel())
    SVRModel = SVRModel.best_estimator_
    y_pred = SVRModel.predict(X_test)
    
    R2_SVR.append(SVRModel.score(X_test, y_test))
    RMSE_SVR.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    scatter_plot('SVR with radial basis function kernel', y_test, y_pred)

    return SVRModel, np.around(np.mean(R2_SVR),decimals=4)*100, np.mean(RMSE_SVR)


def train_gradient_boosting(X_train, y_train, X_test, y_test):
    """ Train a Gradient Boosting Model
    Args:
        X_train (numpy array): input volume data
        y_train (numpy array): input age training data
        X_test (numpy array): volume test data
        y_test (numpy array): age test data
    Returns:
        model (object): A Linear Regression Model object
        R2_GB (float): The model R2 score
        RMSE_GB (float): The model root mean square error
    """
    R2_GB = []
    RMSE_GB = []
    
    regressor = ensemble.GradientBoostingRegressor(random_state=0, loss='ls')
    parameters = {'n_estimators':(100,300,500),'learning_rate':(0.01, 0.1,0.2,0.3,0.4,0.5),'max_depth':(1,3,5,7)}
    scoring_function = make_scorer(mean_squared_error, greater_is_better=False)
    reg = GridSearchCV(regressor, parameters, scoring_function)
    
    reg.fit(X_train, y_train.ravel())
    y_pred = reg.predict(X_test)
    
    #R2_GB.append(reg.score(X_test, y_test))
    R2_GB.append(r2_score(y_train, y_pred))
    RMSE_GB.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    scatter_plot('Gradient boosting', y_test, y_pred)

    return reg.best_estimator_, np.around(np.mean(R2_GB),decimals=4)*100, np.mean(RMSE_GB)


def train_decision_tree(X_train, y_train, X_test, y_test):
    """ Train a Decision Tree Model
    Args:
        X_train (numpy array): input volume data
        y_train (numpy array): input age training data
        X_test (numpy array): volume test data
        y_test (numpy array): age test data
    Returns:
        model (object): A Linear Regression Model object
        R2_DT (float): The model R2 score
        RMSE_DT (float): The model root mean square error
    """
    R2_DT = []
    R2_DT_ada = []
    RMSE_DT = []
    RMSE_DT_ada = []
    
    regressor = tree.DecisionTreeRegressor(max_depth=8)
    regressor_ada = ensemble.AdaBoostRegressor(tree.DecisionTreeRegressor(max_depth=8),
                          n_estimators=300)

    regressor.fit(X_train, y_train.ravel())
    regressor_ada.fit(X_train, y_train.ravel())
    
    y_pred = regressor.predict(X_test)
    y_pred_ada = regressor_ada.predict(X_test)

    R2_DT.append(r2_score(y_train, y_pred))
    R2_DT_ada.append(r2_score(y_train, y_pred_ada))
    
    RMSE_DT.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    RMSE_DT_ada.append(np.sqrt(mean_squared_error(y_test, y_pred_ada)))

    plt.figure()
    plt.title('Decision Tree using Ada Boost', fontsize=13)
    
    # Thanks to https://stackoverflow.com/a/34004236
    plt.rcParams['xtick.labelsize'] = 9
    plt.rcParams['ytick.labelsize'] = 9

    
    plt.scatter(y_train, y_pred)
    plt.scatter(y_train, y_pred_ada, c='xkcd:seafoam')
    
    plt.scatter(y_test, y_test, c='xkcd:bubblegum')
    
    plt.xlabel('True Age', fontsize = 11)
    plt.ylabel('Predicted Age', fontsize = 12)
    
    plt.legend(['Model', 'Model ADA', 'True'], fontsize = 9, loc='upper left')
    plt.show()
    print(np.mean(R2_DT_ada)*100)
    return regressor, np.around(np.mean(R2_DT),decimals=4)*100, np.mean(RMSE_DT)

In [11]:
X_df = pd.read_csv(os.path.join(data_dir, 'features.csv'))
X = np.array(X_df)

# Read labels from meta-data
y = [meta_data['age'][i] for i in range(len(meta_data))]
y = np.reshape(y,(len(meta_data),1))

In [12]:
# Regression time! :) 
# Regressions (Linear, Bayessian Ridge, SVR with RBF)
# Ensure that package is installed
#!pip install tabulate
from tabulate import tabulate


# Subset of X with the total volume
X_sub = X[:, 0:3]

# Test X just for GM and WM
# Note: You can comment out the X_sub above and replace with this 
# Best results though were achieved using the combination of all volumes! 
# X_sub = np.transpose(np.vstack([X[:,1].ravel(), X[:,2].ravel()]))

# Even though it can use the train functions inside one loop
# It's much faster to perform k-fold multiple times
# and create multiple loops

kf = KFold(n_splits=2, random_state = False, shuffle = True)
kf.get_n_splits(X_sub)

for train_index, test_index in kf.split(X_sub):
    X_train, X_test = X_sub[train_index], X_sub[test_index]
    y_train, y_test = y[train_index], y[test_index]   
        
    _, R2L, RMSEL = train_linear_regression(X_train, y_train, X_test, y_test)

kf = KFold(n_splits=2, random_state = False, shuffle = True)
kf.get_n_splits(X_sub)

for train_index, test_index in kf.split(X_sub):
    X_train, X_test = X_sub[train_index], X_sub[test_index]
    y_train, y_test = y[train_index], y[test_index]   
        
    _, R2B, RMSEB = train_bayessian_regression(X_train, y_train, X_test, y_test)

kf = KFold(n_splits=2, random_state = False, shuffle = True)
kf.get_n_splits(X_sub)

for train_index, test_index in kf.split(X_sub):
    X_train, X_test = X_sub[train_index], X_sub[test_index]
    y_train, y_test = y[train_index], y[test_index]   
        
    _, R2_SVR, RMSE_SVR = train_SVR_RBF(X_train, y_train, X_test, y_test)

kf = KFold(n_splits=2, random_state = False, shuffle = True)
kf.get_n_splits(X_sub)

for train_index, test_index in kf.split(X_sub):
    X_train, X_test = X_sub[train_index], X_sub[test_index]
    y_train, y_test = y[train_index], y[test_index]   
        
    _, R2_GB, RMSE_GB = train_gradient_boosting(X_train, y_train, X_test, y_test)
    
kf = KFold(n_splits=2, random_state = False, shuffle = True)
kf.get_n_splits(X_sub)

for train_index, test_index in kf.split(X_sub):
    X_train, X_test = X_sub[train_index], X_sub[test_index]
    y_train, y_test = y[train_index], y[test_index]   

    _, R2_DT, RMSE_DT = train_decision_tree(X_train, y_train, X_test, y_test)

In [13]:
# Display the data values in tabular format
metric_values = [["Linear Regression", R2L, RMSEL], ["Bayessian Regression", R2B, RMSEB], ["SVR with RBF", R2_SVR, RMSE_SVR], ["Gradient Boosting", R2_GB, RMSE_GB], ["Decision Tree", R2_DT, RMSE_DT]]
titles = ["Regression", "r2-score (mean, %)", "RMSE (mean)"]

print(tabulate(metric_values, headers=titles, colalign=("center", "center", "center"), tablefmt="fancy_grid"))

## Part B: Image-based regression using grey matter maps
The second approach will make use of grey matter maps that have been already extracted from the set of MRI scans and aligned to a common reference space to obtain spatially normalised maps. For this, we have used an advanced, state-of-the-art neuroimaging toolkit, called SPM12. The reference space corresponds to the MNI atlas (compare slide 99 of the Segmentation lecture).

Because the grey matter maps are spatially normalised, voxel locations across images from different subjects roughly correspond to the same anatomical locations. This means that each voxel location in the grey matter maps can be treated as an individual feature. Because those maps are quite large, there would be a very large number of features to deal with. A dimensionality reduction using PCA needs to be performed before training a suitable regressor on the low-dimensional feature representation obtained with PCA. It might also be beneficial to apply some pre-processing before running PCA, which should be explored. The implemented pipeline should be evaluated using cross-validation using the same data splits as in part A, so the two approaches can be directly compared.

Note: For part B, only the spatially normalised grey matter maps should be used.

### TASK B-1: Pre-processing

Before running PCA to reduce the dimensionality of the feature space for grey matter maps, it might be beneficial to run some pre-processing on the maps. In voxel-based analysis, such as training a regressor where each voxel location is a feature, it is common to apply some smoothing beforehand. This is to reduce noise and to compensate for errors of the spatial normalisation that had been applied to the maps.

Because the maps are quite large, it might also be worthwile to explore whether downsampling could be performed, before PCA. This would further reduce the dimensionality, and might be even needed in the case where PCA on the orignial resolution runs into memory issues.

Implement a function that performs suitable pre-processing on each grey matter map.

*Hint:* Check out tutorial 1. You may want to save the pre-processed maps using `sitk.WriteImage` to avoid recomputation.

In [14]:
from scipy import signal
from scipy import ndimage
from skimage import transform
import glob

def preprocess_gray_matter(gm, us_factor):
    """ Preprocess each gray matter image by 
    Args:
        gm(sitk image): The graymatter image file
        us_factor(float): An undersampling factor
    Returns:
        The downsampled image using local averaging
    """
    # for a sigma value chosen experimentally
    for i in range(gm.shape[2]):
        gm[:,:,i] = ndimage.gaussian_filter(gm[:,:,i], sigma=0.65)

    return transform.downscale_local_mean(gm, factors = (us_factor,us_factor,us_factor))

def store_preprocessed(us_factor):
    if not os.path.exists(os.path.join(data_dir, 'gm{}/'.format(us_factor))):
        os.mkdir(os.path.join(data_dir, 'gm{}/'.format(us_factor)))
        
        return os.path.join(data_dir, 'gm{}/'.format(us_factor))
    else:    
        return os.path.join(data_dir, 'gm{}/'.format(us_factor))

In [15]:
test_factors = np.arange(1,8)

# plt.rcdefaults()
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_style('white')
# plt.set_cmap('gist_earth')

for each_factor in test_factors: 
    pre_store = store_preprocessed(each_factor)
    for i in range(meta_data['ID'].count()):
        ID = meta_data['ID'][i]
        age = meta_data['age'][i]

        gmd = os.path.join(data_dir, 'greymatter/')
        gm_filenames = glob.glob(gmd + '*.nii.gz')

        gm_filename = [f for f in gm_filenames if ID in f][0]
        gm = sitk.ReadImage(gm_filename)
        #print(gm_filename)
        gm_array = sitk.GetArrayFromImage(gm)

        # process each file with each factor
        gm_smooth_array =  preprocess_gray_matter(gm_array, each_factor)
        gm_smooth = sitk.GetImageFromArray(gm_smooth_array)

        #gm_smooth.CopyInformation(gm) 
        #print(pre_store)
        #print(os.path.join(pre_store, os.path.basename(ID + '.nii.gz')))
        gm_name = os.path.join(pre_store, os.path.basename(ID + '.nii.gz'))

        #display_image(gm)

        #display_image(gm_smooth)
        sitk.WriteImage(gm_smooth, gm_name, True)

In [16]:
test_factors = np.arange(1,3)

# Create a dictionary storing all the features
# Using custom specified test factors

keys = ['X_PCA_'+ str(i) for i in test_factors]
values = [[] for i in test_factors]
feat_dict = dict(zip(keys, values))

i=1
for key, values in feat_dict.items():
    for j in range(meta_data['ID'].count()):
        ID = meta_data['ID'][j]
        age = meta_data['age'][j]

        gm_dir = os.path.join(data_dir, 'gm{}/'.format(i))
        gm_filenames = glob.glob(gm_dir + '*.nii.gz')

        gm_filename = [f for f in gm_filenames if ID in f][0]
        
        gm = sitk.ReadImage(gm_filename)
        gm_array = sitk.GetArrayFromImage(gm)
        
        feat_dict[key].append(np.ndarray.flatten(gm_array))

    scaler = StandardScaler()
    scaler.fit(feat_dict[key])
    X_PCA = scaler.transform(feat_dict[key])
    i += 1

### TASK B-2: Dimensionality reduction

Implement dimensionality reduction for grey matter maps using [scitkit-learn's PCA](http://scikit-learn.org/stable/modules/decomposition.html#pca). PCA has an option to set the percentage of variance that needs to be preserved (by setting the parameter `n_components` to a value between 0 and 1). The number principal modes, that is the new dimensionality of the data, is then automatically determined. Try initially to preserve 95% of the variance (`n_components=0.95`).

*Note:* When dimensionality reduction is used as pre-processing step for supervised learning, as in this case, it is important that PCA is fitted to the training data only, but then applied to both the training and testing data. So make sure your implementation consists of two separate steps, 1) fitting the PCA model to $X_{\text{train}}$ (using the `fit` function), and 2) applying dimensionality reduction to $X_{\text{train}}$ and $X_{\text{test}}$ using the `transform` function.

In [17]:
import sklearn.decomposition as decomp

def PCA(X_train, X_test):
    """ Perform a PCA by preserving the variance percentage to 95%
    Args:
        X_train (np array): The training data
        X_test (np.array): The testing input data
    Returns:
        X_train_pca (np.array): The PCA result on train data
        X_test_pca (np.array): The PCA result on test data
    """
    pca = decomp.PCA(n_components = 0.95)

    # Fit the data
    pca.fit(X_train)

    cum_sum=np.cumsum(pca.explained_variance_ratio_)

    plt.plot(cum_sum)
    plt.xticks(np.arange(0, pca.explained_variance_ratio_.shape[0],5))
    plt.title('PCA eigenvalues cummulative distribution')
    plt.ylabel('Cummulative Sum')
    plt.ylabel('Cummulative Sum')
    
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    return X_train_pca, X_test_pca

### TASK B-3: Age regression and cross-validation

Experiment with different regression methods from the [scikit-learn toolkit](http://scikit-learn.org/stable/supervised_learning.html#supervised-learning). Evaluate the methods using two-fold [cross-validation](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) in the same way as for approach A so results can be directly compared.

Try using at least two different regression methods.

*Note:* Remember, when you use cross-validation where you swap training and testing sets in each fold, you need to fit PCA to the training set of each fold.

In [2]:
#X_sub = X[:,0].reshape(-1,1)
X_sub = np.array(feat_dict['X_PCA_2'])

# Remove comments to run all the regressions

# # Linear Regression
# kf = KFold(n_splits=2, random_state = False, shuffle = True)
# kf.get_n_splits(X_sub)

# for key, values in feat_dict.items():     
#     X_PCA = np.array(feat_dict[key])
#     for train_index, test_index in kf.split(X_sub):
#         X_train, X_test = X_PCA[train_index], X_PCA[test_index]
#         y_train, y_test = y[train_index], y[test_index]   
        
#         X_train_pca, X_test_pca = PCA(X_train, X_test)

#         _, R2L, RMSEL = train_linear_regression(X_train_pca, y_train, X_test_pca, y_test)

# Decision Tree with ADA Boost
# kf = KFold(n_splits=2, random_state = False, shuffle = True)
# kf.get_n_splits(X_sub)

# for key, values in feat_dict.items():     
#     X_PCA = np.array(feat_dict[key])
#     for train_index, test_index in kf.split(X_sub):
#         X_train, X_test = X_PCA[train_index], X_PCA[test_index]
#         y_train, y_test = y[train_index], y[test_index]   
        
#         X_train_pca, X_test_pca = PCA(X_train, X_test)

#         _, R2_DT, RMSE_DT = train_decision_tree(X_train, y_train, X_test, y_test)

# SVR RBF
kf = KFold(n_splits=2, random_state = False, shuffle = True)
kf.get_n_splits(X_sub)

for key, values in feat_dict.items():     
    X_PCA = np.array(feat_dict[key])
    for train_index, test_index in kf.split(X_sub):
        X_train, X_test = X_PCA[train_index], X_PCA[test_index]
        y_train, y_test = y[train_index], y[test_index]   
        
        X_train_pca, X_test_pca = PCA(X_train, X_test)
        
        _, R2_SVR, RMSE_SVR = train_SVR_RBF(X_train, y_train, X_test, y_test)

In [3]:
# Display the data values in tabular format
metric_values = [["Linear Regression", R2L, RMSEL], ["SVR with RBF", R2_SVR, RMSE_SVR], ["Decision Tree", R2_DT, RMSE_DT]]
titles = ["Regression", "r2-score (mean, %)", "RMSE (mean)"]

print(tabulate(metric_values, headers=titles, colalign=("center", "center", "center"), tablefmt="fancy_grid"))